In [1]:
from langchain_community.chat_models import ChatTongyi
from langchain_community.embeddings import DashScopeEmbeddings

In [12]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
api_key = os.environ['DASHSCOPE_API_KEY']

In [13]:
chat = ChatTongyi(model='qwen-long', top_p=0.8, temperature=0, api_key=api_key)

In [14]:
chat.invoke('who are you?')

AIMessage(content="I'm Qwen, a large language model created by Alibaba Cloud. I'm designed to assist users in generating various types of text, such as articles, stories, poems, and answering questions by leveraging the vast amount of information I've been trained on. How can I assist you today?", response_metadata={'model_name': 'qwen-long', 'finish_reason': 'stop', 'request_id': 'c140bf65-1bde-9874-9450-cfa07e3842be', 'token_usage': {'input_tokens': 4, 'output_tokens': 58, 'total_tokens': 62}}, id='run-723cf39d-c29e-4453-a7cb-096d38305c87-0')

In [8]:
from langchain_core.messages import SystemMessage
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

In [9]:
sys_msg = SystemMessage("You are a helpful assistant")

In [10]:
usr_msg = HumanMessage('Give me the answer of 1 + 1 =?')
messages = [sys_msg, usr_msg]

In [14]:
result = chat.invoke(input=messages)

In [15]:
result.content

'1 + 1 = 2'

In [27]:
from langchain_core.prompts import SystemMessagePromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate

In [17]:
role = {'English teacher': 'How to say I am blue in a long sentence',
        'Math teacheer': '1 + 1 = ?'
       }

In [21]:
sys_msg = SystemMessagePromptTemplate.from_template(f'You are an English teacher')
content = role['English teacher']

In [22]:
usr_msg = HumanMessagePromptTemplate.from_template(f'Please answer the question of {content}')

In [24]:
messages = [sys_msg, usr_msg]

In [28]:
prompt = ChatPromptTemplate.from_messages(messages)

In [34]:
formatted_prompt = prompt.format()

In [40]:
chat.invoke(formatted_prompt)

AIMessage(content='Sure, you can express "I am blue" in a longer, more descriptive sentence like this:\n\n"Despite the bright sunshine and the cheerful company around me, I find myself feeling rather blue today, as a wave of melancholy thoughts has taken hold of my mood."', response_metadata={'model_name': 'qwen-long', 'finish_reason': 'stop', 'request_id': '71f1e4ee-03ee-9ca7-aa32-e354f9da8a6c', 'token_usage': {'input_tokens': 25, 'output_tokens': 54, 'total_tokens': 79}}, id='run-ec487f3f-7b04-4703-8ce7-1296155e8830-0')

In [30]:
from langchain_core.output_parsers import StrOutputParser

In [36]:
output_parser = StrOutputParser()

In [41]:
chain = chat | output_parser

In [43]:
chain.invoke(input=formatted_prompt)

'Sure, you can express "I am blue" in a longer, more descriptive sentence like this:\n\n"Despite the bright sunshine and the cheerful company around me, I find myself feeling rather blue today, as a lingering sense of melancholy has settled over my thoughts and emotions."'

### FewShotPromptTemplate:
少样本学习，大模型"能力涌现"的典型代表；大模型本身没有在这个方向上做大量的训练，但可以根据少量的例子，学到背后的知识，从而迅速处理相关的问题

In [26]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate
from langchain_core.prompts import PipelinePromptTemplate

In [16]:
example_prompt = PromptTemplate.from_template(template='City: {city}\Province: {province}')

In [17]:
examples = [{
    'city': 'Xian',
    'province': 'Shanxi'
},
    {
        'city': 'Zibo',
        'province': 'Shandong'
    }]

In [20]:
prompt = FewShotPromptTemplate(example_prompt=example_prompt,
                               examples=examples,
                               prefix='Refer to the examples, and response to the question:\n',
                               suffix='City: {city}\Province')

In [37]:
result = prompt.invoke(input={"city": "ZhengZhou"})

In [38]:
result.text

'List the top 10 population in the world and population'

In [39]:
chat.invoke(result)

SSLError: HTTPSConnectionPool(host='dashscope.aliyuncs.com', port=443): Max retries exceeded with url: /api/v1/services/aigc/text-generation/generation (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1131)')))

In [ ]:
full_prompt = PromptTemplate.from_template(())

### PydanticOutParser
- 对动态语言中的数据类型，进行约束，类似ts

In [27]:
from langchain.output_parsers import PydanticOutputParser

In [29]:
from langchain.pydantic_v1 import BaseModel, Field

In [30]:
class ResultEntity(BaseModel):
    country: str = Field(description='Country')
    population: str = Field(description='Population in the related country')

In [32]:
parser = PydanticOutputParser(pydantic_object=ResultEntity)

In [33]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"country": {"title": "Country", "description": "Country", "type": "string"}, "population": {"title": "Population", "description": "Population in the related country", "type": "string"}}, "required": ["country", "population"]}\n```'

In [40]:
format_str = parser.get_format_instructions()
prompt = PromptTemplate.from_template(template='List the top 10 population in the world and population', partial_variables={'format': format_str})

In [41]:
chain = prompt | chat

In [43]:
chain.invoke(input={})

AIMessage(content='Sure, here are the top 10 most populous countries in the world along with their approximate populations as of 2023:\n\n1. **China** - Approximately 1.4 billion people.\n2. **India** - Approximately 1.4 billion people (expected to surpass China in population in 2023).\n3. **United States** - Approximately 332 million people.\n4. **Indonesia** - Approximately 276 million people.\n5. **Pakistan** - Approximately 225 million people.\n6. **Brazil** - Approximately 214 million people.\n7. **Nigeria** - Approximately 211 million people.\n8. **Bangladesh** - Approximately 166 million people.\n9. **Russia** - Approximately 145 million people.\n10. **Mexico** - Approximately 130 million people.\n\nPlease note that these numbers are estimates and can vary slightly depending on the source.', response_metadata={'model_name': 'qwen-long', 'finish_reason': 'stop', 'request_id': '74b336f2-7b41-960a-86ea-26ad4eb08214', 'token_usage': {'input_tokens': 12, 'output_tokens': 202, 'total_